In [1]:
import pandas as pd


In [4]:
df = pd.read_parquet("train.parquet")  # If using local file

In [20]:
for i in range(len(df))[:100]:
    english = df.iloc[i].iloc[0]['en']
    deutsch = df.iloc[i].iloc[0]['de']
    print("------------------------")
    print("EN: ", english)
    print("DE: ", deutsch)

------------------------
EN:  It's greed that it's gonna be the death of you, 'cause you...
DE:  Deine Habgier wird noch dein Tod sein.
------------------------
EN:  Vega.
DE:  - Vega.
------------------------
EN:  Just say when.
DE:  Sagen Sie einfach stopp.
------------------------
EN:  - Wait.
DE:  - Warte.
------------------------
EN:  I don't wanna be here.
DE:  Ich will nicht hier sein.
------------------------
EN:  Okay, so 90 percent of my photographic process is, in fact, not photographic.
DE:  Also, 90 Prozent meiner fotografischen Arbeit ist genau genommen gar nicht fotografisch.
------------------------
EN:  So lovely.
DE:  Wie bezaubernd.
------------------------
EN:  Thank you, colonel.
DE:  Vielen Dank, Colonel.
------------------------
EN:  Martin!
DE:  Martin!
------------------------
EN:  Do I have to?
DE:  Muss ich?
------------------------
EN:  Please sit.
DE:  Nehmen Sie Platz.
------------------------
EN:  Leo!
DE:  Leo!
------------------------
EN:  No, honestly,